# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mataura,-46.1927,168.8643,14.37,90,99,4.49,NZ,1675620689
1,1,hilo,19.7297,-155.0900,23.49,95,100,2.57,US,1675620600
2,2,venice,27.0998,-82.4543,27.93,76,75,4.12,US,1675620690
3,3,flinders,-34.5833,150.8552,20.66,80,96,1.34,AU,1675620691
4,4,punta arenas,-53.1500,-70.9167,12.06,43,0,15.95,CL,1675620638


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Lat"]>= -50) & (city_data_df["Lat"]<= 40) & (city_data_df["Cloudiness"]<= 25)& (city_data_df["Max Temp"]>= 25)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,rikitea,-23.1203,-134.9692,25.91,75,9,7.14,PF,1675620703
33,33,atuona,-9.8000,-139.0333,25.72,78,11,7.54,PF,1675620589
41,41,pisco,-13.7000,-76.2167,25.03,78,0,4.63,PE,1675620715
48,48,kavaratti,10.5669,72.6420,27.18,74,6,3.62,IN,1675620660
49,49,valparaiso,-33.0393,-71.6273,25.13,56,0,4.63,CL,1675620061
71,71,vila velha,-20.3297,-40.2925,30.97,74,20,7.20,BR,1675620518
74,74,santa maria,-29.6842,-53.8069,29.78,32,18,2.43,BR,1675620758
83,83,mar del plata,-38.0023,-57.5575,27.23,41,17,4.89,AR,1675620646
116,116,tandil,-37.3217,-59.1332,32.05,25,21,4.62,AR,1675620805
134,134,panama city,8.9936,-79.5197,32.97,48,20,7.72,PA,1675620563


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country',"Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
21,rikitea,PF,-23.1203,-134.9692,75,
33,atuona,PF,-9.8000,-139.0333,78,
41,pisco,PE,-13.7000,-76.2167,78,
48,kavaratti,IN,10.5669,72.6420,74,
49,valparaiso,CL,-33.0393,-71.6273,56,
71,vila velha,BR,-20.3297,-40.2925,74,
74,santa maria,BR,-29.6842,-53.8069,32,
83,mar del plata,AR,-38.0023,-57.5575,41,
116,tandil,AR,-37.3217,-59.1332,25,
134,panama city,PA,8.9936,-79.5197,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row['Lat']
    Lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{Lat},{Lng},{radius}"
    params["bias"] = f"proximity:{Lat},{Lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    requests.adapters.DEFAULT_RETRIES = 5
    name_address = requests.get(base_url, params=params) # increase retries number
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError,):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rikitea - nearest hotel: No hotel found
atuona - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
santa maria - nearest hotel: No hotel found
mar del plata - nearest hotel: No hotel found
tandil - nearest hotel: No hotel found
panama city - nearest hotel: No hotel found
cidreira - nearest hotel: No hotel found
souillac - nearest hotel: No hotel found
caravelas - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
acapulco - nearest hotel: No hotel found
arlit - nearest hotel: No hotel found
dulce nombre de culmi - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
chuy - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
bambous virieux - nearest hotel: No hotel found
vallenar - nearest hotel: No hotel found
alappuzha - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
21,rikitea,PF,-23.1203,-134.9692,75,No hotel found
33,atuona,PF,-9.8000,-139.0333,78,No hotel found
41,pisco,PE,-13.7000,-76.2167,78,No hotel found
48,kavaratti,IN,10.5669,72.6420,74,No hotel found
49,valparaiso,CL,-33.0393,-71.6273,56,No hotel found
71,vila velha,BR,-20.3297,-40.2925,74,No hotel found
74,santa maria,BR,-29.6842,-53.8069,32,No hotel found
83,mar del plata,AR,-38.0023,-57.5575,41,No hotel found
116,tandil,AR,-37.3217,-59.1332,25,No hotel found
134,panama city,PA,8.9936,-79.5197,48,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
#hotel_df.loc[hotel_df['Hotel Name']!= "No hotel found"]

hot_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    hover_cols=["Hotel Name", "Country"],
    color = "City"
)

# Display the map
hot_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)